In [1]:
# INSTALLATION OF PACKAGES

import pandas as pd                 # Data manipulation and analysis
import matplotlib.pyplot as plt     # Plotting and visualization
import seaborn as sns               # Statistical data visualization
import numpy as np                  # Numerical operations
import geopandas as gpd             # Geospatial data manipulation
import ipywidgets as widgets        # Interactive widgets for Jupyter notebooks

import os                           # Operating system interfaces
import requests                     # HTTP library for making requests
import warnings                     # Warning control

Se descarga la base de datos y se guarda en una carpeta llamada Datos

In [2]:
# URL of the database
file_download_link = "https://drive.usercontent.google.com/download?id=14cr_NcbnRlxGBAE2KwluXO5RWcV7h27a&export=download&authuser=0&confirm=t&uuid=ab338839-bb46-415c-9e0b-baa349f66125&at=ALWLOp58TXtgKEDFavl5Kn2Hru6H:1762816912784"

# Folder creation and path definition
folder_name = "Datos"
file_name = "RBDSimce4toBasico2014-2024.xlsx"

# Create the folder if it does not exist
os.makedirs(folder_name, exist_ok=True)
file_path = os.path.join(folder_name, file_name) # Full file path

# Download and save the file
response = requests.get(file_download_link)
with open(file_path, 'wb') as f:
    f.write(response.content)

print(f"File downloaded successfully at: {file_path}")

File downloaded successfully at: Datos\RBDSimce4toBasico2014-2024.xlsx


al cargar los datos, se ignoran las advertencias ya que algunas celdas pueden tener formatos inconsistentes
provocando advertencias que no afectan el análisis de los datos.

In [3]:
#load data from the SIMCE 4th grade results from 2014 to 2024

warnings.simplefilter("ignore")

#loads data 
df_simce_2014 = pd.read_excel('Datos/RBDSimce4toBasico2014-2024.xlsx', sheet_name='2014')
df_simce_2015 = pd.read_excel('Datos/RBDSimce4toBasico2014-2024.xlsx', sheet_name='2015')
df_simce_2016 = pd.read_excel('Datos/RBDSimce4toBasico2014-2024.xlsx', sheet_name='2016')
df_simce_2017 = pd.read_excel('Datos/RBDSimce4toBasico2014-2024.xlsx', sheet_name='2017')
df_simce_2018 = pd.read_excel('Datos/RBDSimce4toBasico2014-2024.xlsx', sheet_name='2018')
df_simce_2022 = pd.read_excel('Datos/RBDSimce4toBasico2014-2024.xlsx', sheet_name='2022')
df_simce_2023 = pd.read_excel('Datos/RBDSimce4toBasico2014-2024.xlsx', sheet_name='2023') 
df_simce_2024 = pd.read_excel('Datos/RBDSimce4toBasico2014-2024.xlsx', sheet_name='2024') 

print("Data loaded successfully.")

Data loaded successfully.


Se unifica los DataFrames anuales en un único conjunto consolidado (`df_simce_all_years`). Dado que las columnas pueden variar entre años (nuevas métricas o cambios de metodología), el código realiza una **normalización de esquema** automática.

In [4]:
# Primero se organizan los datos en un diccionario
dfs_dict = {
    '2014': df_simce_2014, '2015': df_simce_2015, '2016': df_simce_2016, 
    '2017': df_simce_2017, '2018': df_simce_2018, '2022': df_simce_2022, 
    '2023': df_simce_2023, '2024': df_simce_2024
}

# Se calculan las columnas comunes
common_columns = set.intersection(*[set(df.columns) for df in dfs_dict.values()])
print(f'Number of common columns: {len(common_columns)}')


# Se verifica si todas las dataframes tienen las mismas columnas y se concatenan para crear un solo dataframe
columns_match = all(len(df.columns) == len(common_columns) for df in dfs_dict.values())


if columns_match: # si todas las columnas coinciden, se concatenan directamente
    df_simce_all_years = pd.concat(dfs_dict.values(), ignore_index=True)
    print('\n Filtered and merged dataframes successfully.')
    
else:  # si no coinciden, se informa de las diferencias y se filtran antes de concatenar
    for year, df in dfs_dict.items():
        diff = set(df.columns) - common_columns
        if diff:
            print(f'Columns in {year} not in common: {diff}')
    df_simce_all_years = pd.concat([df[list(common_columns)] for df in dfs_dict.values()], ignore_index=True)
    print('\n Filtered and merged dataframes successfully.')

Number of common columns: 38
Columns in 2014 not in common: {'difgru_soc4b_rbd', 'prom_soc4b_rbd', 'nom_reg_rbd_2014', 'cod_com_rbd_2014', 'marca_eda_soc4b_rbd', 'palu_eda_ins_soc4b_rbd', 'marca_soc4b_rbd', 'nom_pro_rbd_2014', 'fecha_bbdd', 'nom_com_rbd_2014', 'nalu_soc4b_rbd', 'nom_deprov_rbd_2014', 'marcadif_soc4b_rbd', 'dif_soc4b_rbd', 'sigdif_soc4b_rbd', 'cod_reg_rbd_2014', 'palu_eda_ade_soc4b_rbd', 'nom_deprov_rbd', 'cod_pro_rbd_2014', 'codigo_bbdd', 'siggru_soc4b_rbd', 'marca_eda_lect4b_rbd', 'palu_eda_ele_soc4b_rbd', 'marca_eda_mate4b_rbd'}
Columns in 2015 not in common: {'fecha_bbdd', 'nom_deprov_rbd', 'cod_deprov_rbd', 'codigo_bbdd'}
Columns in 2016 not in common: {'fecha_bbdd', 'nom_deprov_rbd', 'codigo_bbdd'}
Columns in 2018 not in common: {'fecha_bbdd', 'nom_deprov_rbd', 'cod_deprov_rbd', 'codigo_bbdd'}
Columns in 2022 not in common: {'fecha_bbdd', 'nom_deprov_rbd', 'cod_deprov_rbd', 'codigo_bbdd'}
Columns in 2023 not in common: {'fecha_bbdd', 'nom_deprov_rbd', 'cod_deprov_

In [5]:
df_simce_all_years.describe()

,prom_mate4b_rbd,nalu_mate4b_rbd,dvrbd,cod_com_rbd,cod_reg_rbd,difgru_mate4b_rbd,palu_eda_ele_mate4b_rbd,dif_lect4b_rbd,palu_eda_ele_lect4b_rbd,palu_eda_ins_mate4b_rbd,prom_lect4b_rbd,nalu_lect4b_rbd,cod_pro_rbd,rbd,difgru_lect4b_rbd,palu_eda_ade_mate4b_rbd,dif_mate4b_rbd,palu_eda_ade_lect4b_rbd,palu_eda_ins_lect4b_rbd,agno
count,54820.000000,58487.000000,59189.000000,59189.000000,59189.000000,49428.000000,44360.000000,48101.000000,44328.000000,44360.000000,54856.000000,58487.000000,59189.000000,59189.000000,49416.000000,44360.000000,48119.000000,44328.000000,44328.000000,59189.000000
mean,244.972601,29.451451,4.480647,6924.199294,9.005643,-1.899672,34.287193,1.556392,26.191565,37.881506,258.037571,29.274163,47.044518,9610.434152,-1.196596,18.854612,0.982647,34.798946,29.960959,2018.543513
std,44.664989,30.342481,2.881179,5106.261046,3.611760,20.248251,15.888895,19.208006,12.511650,23.871446,44.608976,30.130788,52.677725,8201.773237,18.574622,17.755270,19.037830,20.736994,19.483667,3.594657
min,0.000000,0.000000,0.000000,1.000000,1.000000,-141.000000,0.000000,-118.000000,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,-133.000000,0.000000,-118.000000,0.000000,0.000000,2014.000000
25%,230.000000,6.000000,2.000000,48.000000,6.000000,-14.000000,26.500000,-9.000000,20.000000,19.000000,245.000000,6.000000,1.000000,3448.000000,-12.000000,5.600000,-9.000000,20.500000,15.400000,2015.000000
50%,249.000000,21.000000,4.000000,7401.000000,9.000000,0.000000,36.800000,0.000000,27.500000,38.500000,263.000000,21.000000,7.000000,7496.000000,0.000000,14.300000,0.000000,34.400000,29.000000,2017.000000
75%,268.000000,40.000000,7.000000,11101.000000,13.000000,10.000000,45.100000,12.000000,34.000000,55.600000,280.000000,40.000000,91.000000,12682.000000,9.000000,27.500000,11.000000,48.700000,43.200000,2022.000000
max,376.000000,282.000000,9.000000,16305.000000,16.000000,108.000000,85.700000,137.000000,80.000000,100.000000,396.000000,284.000000,163.000000,42261.000000,124.000000,100.000000,113.000000,100.000000,100.000000,2024.000000


In [6]:
df_simce_all_years.rename(columns={'nom_reg_rbd': 'Region', 'nom_com_rbd': 'Comuna', 'prom_lect4b_rbd': 'Avg Reading Score'}, inplace=True)
df_simce_all_years['Region']

0        REGIÓN ARICA - PARINACOTA
1        REGIÓN ARICA - PARINACOTA
2        REGIÓN ARICA - PARINACOTA
3        REGIÓN ARICA - PARINACOTA
4        REGIÓN ARICA - PARINACOTA
                   ...            
59184              DE LA ARAUCANÍA
59185                   DEL BIOBÍO
59186                   DEL BIOBÍO
59187    DEL LIBERTADOR BERNARDO O
59188    DEL LIBERTADOR BERNARDO O
Name: Region, Length: 59189, dtype: object

In [7]:
#gdf_regions = gpd.read_file('Regional.shp',encoding='utf-8')
gdf_cities = gpd.read_file('comunas.shp', encoding='utf-8')

# Example workflow to select a specific region and its cities from shapefiles

# --- 1) Select the region you want (by code or name) ---
# Example: by code
target_name = "Región Metropolitana de Santiago"  # Región Metropolitana de Santiago, adjust to yours
#region = gdf_regions.loc[gdf_regions["Region"] == target_name].copy()

# --- 2) Filter cities that lie inside the selected region ---
#Based on the name of the Region column in the cities shapefile
cities_in_region = gdf_cities[gdf_cities['Region'] == target_name]

# --- 3) Plot the region and its cities; zoom to the region extent ---
fig, ax = plt.subplots(figsize=(7, 7))
#region.plot(ax=ax, color="#e8eff7", edgecolor="black", linewidth=1)
cities_in_region.plot(ax=ax, markersize=15)
#print how many cities were found in the region
print(f'Number of cities found in {target_name}: {len(cities_in_region)}')

# Zoom to region with a small padding
#minx, miny, maxx, maxy = region.total_bounds
#pad_x = (maxx - minx) * 0.05
#pad_y = (maxy - miny) * 0.05
#ax.set_xlim(minx - pad_x, maxx + pad_x)
#ax.set_ylim(miny - pad_y, maxy + pad_y)

#addd the title
ax.set_axis_off()
ax.set_title("Cities within the selected region", pad=10)
plt.tight_layout()
plt.show()

DataSourceError: comunas.shp: No such file or directory